# Training a word embedding model from scratch

This notebook illustrates how to use `gensim` to train a word2vec model from scratch on original data.

## Setup

In [1]:
import os
import gensim
import numpy as np
import pandas as pd

## Load the data

In [2]:
data_path = os.path.join('..', 'data', 'corpora', 'gbr_commons')
os.makedirs(data_path, exist_ok=True)

I have prepared a corpus of sentence-splitted speeches from the UK *House of Commons* (lower house chamber of the parliament).
The file is too big to be uploaded on Github.
So you need to download it if you have not yet done so:

In [3]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
if not os.path.exists(fp):
    print('downloading the corpus ... might take 3-10 minutes (depending on your internet connection)')
    url = 'https://www.dropbox.com/scl/fi/wkxj7k2uiy0935dmbjp34/gbr_commons_speech_sentences_tokenized.tsv.gzip?rlkey=urjdpz0vgbymllzugzsh0m2ld&dl=1'
    corp = pd.read_csv(url, sep='\t', compression='gzip')
    corp.to_csv(fp, sep='\t', compression='gzip', index=False)
    # keep only the first 100K sentences in the corpus
    corp = corp.iloc[:100_000]
else:
    # load only the first 100K sentences in the corpus
    corp = pd.read_csv(fp, sep='\t', compression='gzip', nrows=100_000)

corp = corp[~corp.text_tokenized.isna()]

In [4]:
corp.head()

,text_id,text_tokenized
0,uk.org.publicwhip/debate/1970-01-19a.1.5_0_0,On a point of order
1,uk.org.publicwhip/debate/1970-01-19a.1.5_0_1,Is it true Mr Speaker that your Private Secret...
2,uk.org.publicwhip/debate/1970-01-19a.1.5_0_2,If that be true then it is a matter of deep re...
3,uk.org.publicwhip/debate/1970-01-19a.1.5_0_3,Will you Mr Speaker be kind enough to convey o...
4,uk.org.publicwhip/debate/1970-01-19a.1.7_0_0,On behalf of hon and right hon Members on this...


The data file maps sentences to sentence IDs.
The sentences have been preprocessed and tokenized into words and then concatenated with white spaces.
So to get a sentences words, we can split at the white space:

In [5]:
# pre-tokenized text
corp.text_tokenized[1]

'Is it true Mr Speaker that your Private Secretary Sir Francis Reid died suddenly over the weekend'

In [6]:
# tokens
print(corp.text_tokenized[1].split())

['Is', 'it', 'true', 'Mr', 'Speaker', 'that', 'your', 'Private', 'Secretary', 'Sir', 'Francis', 'Reid', 'died', 'suddenly', 'over', 'the', 'weekend']


## How to train (illustration)

To train a word2vec model on a corpus, we need to

1. create a new model instance `model = gensim.models.Word2Vec(...)`. Here we pass all model *hyper-parameters* like the embedding dimension and window size, the algorithm we want to use (Skip-gram or CBOW), etc.
2. build the model's vocabulary on our corpus `model.build_vocab(...)`
3. train the model by calling `model.train(...)`

### Train on a corpus (in memory)

All you need as **input** for these steps is a list of sentences split into tokens ("words").
That is, you input data should be prepared as follows:

```python
[
    ['Words', 'in', 'sentence', 'one'],
    ['Some', 'more', 'words', 'in', 'sentence', 'two'],
    ...
]
```

Let's split all our German *Bundestag* speech sentences into words:


In [7]:
# split pre-tokeniezd sentences into words
sentences = corp.text_tokenized.apply(lambda x: x.split(' ')).to_list()
# note: for larger files you can use `sentences = corp.text_tokenized.swirfter.apply(lambda x: x.split(' ')).to_list()` (but `pip install swifter` first)

In [8]:
# print first 4 sentences
sentences[:4]

[['On', 'a', 'point', 'of', 'order'],
 ['Is',
  'it',
  'true',
  'Mr',
  'Speaker',
  'that',
  'your',
  'Private',
  'Secretary',
  'Sir',
  'Francis',
  'Reid',
  'died',
  'suddenly',
  'over',
  'the',
  'weekend'],
 ['If',
  'that',
  'be',
  'true',
  'then',
  'it',
  'is',
  'a',
  'matter',
  'of',
  'deep',
  'regret',
  'to',
  'all',
  'Members',
  'on',
  'both',
  'sides',
  'of',
  'the',
  'House',
  'who',
  'shared',
  'deep',
  'friendships',
  'with',
  'Sir',
  'Francis'],
 ['Will',
  'you',
  'Mr',
  'Speaker',
  'be',
  'kind',
  'enough',
  'to',
  'convey',
  'our',
  'deep',
  'sympathy',
  'to',
  'the',
  'members',
  'of',
  'his',
  'family']]

Now we are ready to create a `Word2Vec` model instance.
The following model hyper-parameters are relevant for our purposes:

- `vector_size`: the number of dimensions $d$ of th word embedding matrix
- `window`: the number of context words to use for modeling 
- `min_count`: the minimum number of times a word needs to occur in the corpus to be included in the vocabulary
- `sg`: set to 1 to use the Skip-Gram algorithm, otherwise uses CBOW algorithm
- `hs`: set to 1 to use the hierarchical softmax
- `negative`: number of negative samples to use per focus word when computing the loss
- `epochs`: number of times to iterate over corpus (all sentences)
- `workers`: number of CPU cores to use for training parallelization

In [9]:
import gensim

# create a new model instance
model = gensim.models.Word2Vec(
    vector_size=10, # <= super low dimensionality for testing
    window=5,
    min_count=5,
    sg=1,
    hs=1,
    negative=10, # <== if negative = 2*window, data is "balanced"
    epochs=5, # <= set very low for testing
    workers=10 # <= 10 because I have 10 cores on my machine
)

In [10]:
model.epochs

5

Next, we build the vocabulary:

In [11]:
# build the vocabulary from the list of sentences
model.build_vocab(sentences)

In [12]:
model.corpus_count

99997

Now we can train the model:

In [13]:
# train the model on the list of sentences
model.train(
    sentences, 
    total_examples=model.corpus_count, 
    epochs=model.epochs
)

(7100376, 10061760)

In [14]:
type(model), type(model.wv)

(gensim.models.word2vec.Word2Vec, gensim.models.keyedvectors.KeyedVectors)

In [15]:
model.wv.vector_size

10

Let's compute similarities on a number of hand-picked word pairs to see whether the model has learned sensible word vectors.

*Note:* This is more of a "face validity" check and no full-fledged validation

In [16]:
# evaluate
pairs = [
    ('Sir', 'Madam'),
    ('Pound', 'Euro'),
    ('Pound', 'Dollar'),
    ('Government', 'law'),
    ('Government', 'bill'),
    ('Government', 'propoasal'),
    ('Opposition', 'law'),
    ('Opposition', 'bill'),
    ('Opposition', 'proposal'),
    ('bill', 'proposal'),
    ('Labour', 'Government'),
    ('Labour', 'Opposition'),
    ('Conservatives', 'Government'),
    ('Conservatives', 'Opposition'),
    ('shadow', 'Minister'),
    ('Opposition', 'bench'),
    ('chair', 'bench'),
    ('Chair', 'Speaker'),
    ('hon', 'Member'),
    ('honourable', 'Member'),
    ('hon', 'Friend'),
    ('Friend', 'Member')
]
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Government" - "law": 0.429
"Government" - "bill": 0.379
"Opposition" - "law": 0.338
"Opposition" - "bill": 0.267
"Opposition" - "proposal": 0.748
"bill" - "proposal": 0.457
"Labour" - "Government": 0.777
"Labour" - "Opposition": 0.721
"Conservatives" - "Government": 0.632
"Conservatives" - "Opposition": 0.714
"shadow" - "Minister": 0.720
"Opposition" - "bench": 0.770
"chair" - "bench": 0.631
"Chair" - "Speaker": 0.951
"hon" - "Member": 0.518
"honourable" - "Member": 0.319
"hon" - "Friend": 0.626
"Friend" - "Member": 0.891


In [17]:
'Minister' in model.wv 

True

### Train with a corpus read from disk

The above code works just fine.
But it has a computational bottleneck!
To iterate over the list of sentences, we need to have them all loaded into our working memory (RAM).
If our corpus is very large -- which it should be to enable learning of reliable word embeddings --, this might be too burdensome for your computer.

So instead of loading the entire corpus in the RAW, we can iteratively read batches of sentences from a data file that exists somewhere on the hard drive ("disk") of your computer.
For this, we need an "iterable" class that implements "yields" sentences in our corpus one at a time.

In the next code cell, we define such a class.
This class reads the sentences from a (ZIP-ed) CSV file and splits sentences at white spaces before yielding them.


In [18]:
import gzip
from tqdm.auto import tqdm

# get corpus sentences
class SentenceCorpus(object):
    """Iterable class for reading a corpus from a file.
    
    Parameters
    ----------
    filepath : str
        Path to the corpus file.
    sep : str, optional
        Separator for splitting the lines into tokens. Default: ','
    compressed : bool, optional
        Whether the file is compressed. Default: False
    skip : int, optional
        Number of lines to skip at the beginning of the file. Default: 1
    nrows : int, optional
        Number of lines to read from the file. Default: None (read all lines)
    
    Yields
    ------
    list of str

    """
    # note: I've already added a lot of functionality here (e.g. reading from compressed files, skip rows, limit the number of rows to read).
    #       Hence, the actual code is a bit more complex than what you'd need in some use cases.
    def __init__(self, filepath, sep='\t', compressed=False, skip=1, nrows=None):
        self.filepath = filepath
        self.sep = sep
        self.compressed = compressed
        # number of lines to skip (e.g., 1 for csv file)
        self.skip = skip if skip else 0
        with gzip.open(self.filepath, 'rt', encoding='utf-8') if self.compressed else open(self.filepath, 'r', encoding='utf-8') as file:
            # count numnber of lines in file
            corpus_size = len(file.readlines())
        self.nrows = nrows if nrows else corpus_size
        if self.skip > 0:
            self.nrows = min(self.nrows+self.skip, corpus_size)
        self.corpus_size = self.nrows-self.skip
    
    def __len__(self):
        return self.corpus_size
    
    def __iter__(self):
        # open file
        with gzip.open(self.filepath, 'rt', encoding='utf-8') if self.compressed else open(self.filepath, 'r', encoding='utf-8') as file:
            # compute how many rows to read in total
            max_ = self.nrows
            # for each line in file
            for i, line in enumerate(file):
                # skip if you want to skipp first `skip` lines
                if i <= self.skip:
                    continue
                # stop if you have read max number of lines
                if i >= max_:
                    break
                # try to split text into words (assuming that the text is in the second column)
                try:
                    # notes: 
                    #   - the `self.sep` says what character you want to use for spliting the line into columns
                    #   - the `[1]` just says that you'll assume that the text is in the second column
                    #     CAUTION: here we assume that the text is in the second column!
                    words = line.strip().split(self.sep)[1].split(' ')
                except:
                    continue
                yield words

Let's read the some 100,000 sentences from our corpus as above:

In [19]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
sentences = SentenceCorpus(filepath=fp, compressed=True, nrows=100_000)
# note: loading it takes time because it needs to count all lines in the file once

In [20]:
len(sentences)

100000

In [21]:
# note: some lines are skipped due to text file formating errors
sum(1 for line in sentences)

99996

In [22]:
model = gensim.models.Word2Vec(
    vector_size=10, # <= super low dimensionality for testing
    window=5,
    min_count=5,
    sg=1,
    hs=1,
    negative=5,
    epochs=5, # <= set very low for testing
    workers=10
)

In [23]:
model.build_vocab(sentences)

In [24]:
model.train(
    sentences, 
    total_examples=model.corpus_count, 
    epochs=model.epochs
)

(7101307, 10061675)

Let's again compute similarities on our hand-picked word pairs:

In [25]:
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Government" - "law": 0.420
"Government" - "bill": 0.400
"Opposition" - "law": 0.375
"Opposition" - "bill": 0.271
"Opposition" - "proposal": 0.708
"bill" - "proposal": 0.373
"Labour" - "Government": 0.791
"Labour" - "Opposition": 0.691
"Conservatives" - "Government": 0.588
"Conservatives" - "Opposition": 0.593
"shadow" - "Minister": 0.669
"Opposition" - "bench": 0.820
"chair" - "bench": 0.683
"Chair" - "Speaker": 0.965
"hon" - "Member": 0.473
"honourable" - "Member": 0.244
"hon" - "Friend": 0.599
"Friend" - "Member": 0.869


## Train, for real

In [26]:
fp = os.path.join(data_path, 'gbr_commons_speech_sentences_tokenized.tsv.gzip')
sentences = SentenceCorpus(filepath=fp, compressed=True)
# number of sentences in the corpus
len(sentences)

26290959

In [27]:
# number of tokens in the corpus
sum([len(text) for text in sentences])

519617649

In [28]:
model = gensim.models.Word2Vec(
    vector_size=100, # <= low dimensionality for testing
    window=5,
    min_count=25, # <= words should appear at least 25 times in the corpus
    sg=1,
    hs=1,
    negative=5,
    epochs=10, # <= set very low for testing
    workers=10,
    sorted_vocab=1,
)

In [29]:
model.build_vocab(sentences) # <== takes long because corpus is large 

Because training is going to take a while, we'll add a "callback" function that prints the loss (i.e., the gap between predicted and observed values) after each epoch

*source:* https://stackoverflow.com/a/54891714

In [30]:
import time
import gensim

class callback(gensim.models.callbacks.CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        # record the start time of the training
    
    def on_epoch_begin(self, model):
        print(f'Epoch #{self.epoch} start ...', end=' ')
        self.ts = time.time()

    def on_epoch_end(self, model):
        # compute the time delta of the epoch relative to self.ts
        delta = time.time() - self.ts
        # print the delta time and loss after each epoch
        loss = model.get_latest_training_loss()
        print(f'Epoch #{self.epoch} took {int(delta // 60)}:{int(delta % 60)}m; loss = {loss}')
        self.epoch += 1

In [32]:
model.train(
    sentences, 
    total_examples=model.corpus_count, 
    epochs=model.epochs,
    # below the thing you need to adapt to use our custom callback that prints progress
    compute_loss=True, 
    callbacks=[callback()]
)

# note: the 'loss' values printed below are increasing =(
#       This is because gensim's reporting is buggy (https://github.com/RaRe-Technologies/gensim/pull/2135)
#       So don't make hard jdugements about 'convergence' 
#        based on the loss values printed below

In [33]:
# Let's again compute similarities on our hand-picked word pairs:
for pair in pairs:
    if all(p in model.wv.key_to_index for p in pair):
        print(f'"{pair[0]}" - "{pair[1]}": {model.wv.similarity(pair[0], pair[1]):.3f}')

"Sir" - "Madam": 0.179
"Pound" - "Euro": 0.235
"Pound" - "Dollar": 0.342
"Government" - "law": 0.160
"Government" - "bill": 0.167
"Opposition" - "law": 0.058
"Opposition" - "bill": 0.147
"Opposition" - "proposal": 0.355
"bill" - "proposal": 0.233
"Labour" - "Government": 0.251
"Labour" - "Opposition": 0.404
"Conservatives" - "Government": 0.302
"Conservatives" - "Opposition": 0.376
"shadow" - "Minister": 0.324
"Opposition" - "bench": 0.190
"chair" - "bench": 0.251
"Chair" - "Speaker": 0.685
"hon" - "Member": 0.258
"honourable" - "Member": 0.119
"hon" - "Friend": 0.342
"Friend" - "Member": 0.558


## Saving the model and it's vectors

In [34]:
# store the model's word vectors in a file
models_path = data_path.replace('corpora', 'models')
os.makedirs(models_path, exist_ok=True)
print(models_path)
fp = os.path.join(models_path, 'gbr_commons_word2vec_w5_d100')
fp

../data/models/gbr_commons


'../data/models/gbr_commons/gbr_commons_word2vec_w5_d100'

With a trained model at hand, you have two options for saving it for re-use.
You can only save the word vectors (as `KeyedVectors`).
Or you can save the full model (which inlcudes the word vectors but has more capabilities).

The following table compares the **pros and cons** of these approaches ([source](https://radimrehurek.com/gensim/models/keyedvectors.html#how-to-obtain-word-vectors)):

<table class="docutils align-default">
<colgroup>
<col style="width: 24%">
<col style="width: 12%">
<col style="width: 11%">
<col style="width: 54%">
</colgroup>
<tbody>
<tr class="row-odd">
<td><p><em>capability</em></p></td>
<td><p><em>KeyedVectors</em></p></td>
<td><p><em>full model</em></p></td>
<td><p><em>note</em></p></td>
</tr>
<tr class="row-even"><td><p>continue training vectors</p></td>
<td><p>❌</p></td>
<td><p>✅</p></td>
<td><p>You need the full model to train or update vectors.</p></td>
</tr>
<tr class="row-odd"><td><p>smaller objects</p></td>
<td><p>✅</p></td>
<td><p>❌</p></td>
<td><p>KeyedVectors are smaller and need less RAM, because they
don’t need to store the model state that enables training.</p></td>
</tr>
<tr class="row-even"><td><p>save/load from native
fasttext/word2vec format</p></td>
<td><p>✅</p></td>
<td><p>❌</p></td>
<td><p>Vectors exported by the Facebook and Google tools
do not support further training, but you can still load
them into KeyedVectors.</p></td>
</tr>
<tr class="row-odd"><td><p>append new vectors</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Add new-vector entries to the mapping dynamically.</p></td>
</tr>
<tr class="row-even"><td><p>concurrency</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Thread-safe, allows concurrent vector queries.</p></td>
</tr>
<tr class="row-odd"><td><p>shared RAM</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Multiple processes can re-use the same data, keeping only
a single copy in RAM using
<a class="reference external" href="https://en.wikipedia.org/wiki/Mmap">mmap</a>.</p></td>
</tr>
<tr class="row-even"><td><p>fast load</p></td>
<td><p>✅</p></td>
<td><p>✅</p></td>
<td><p>Supports <a class="reference external" href="https://en.wikipedia.org/wiki/Mmap">mmap</a>
to load data from disk instantaneously.</p></td>
</tr>
</tbody>
</table>

#### Saving the full model

Just call the save `method()` on your word2vec `mode` object

**_Note:_** It's customary to name the file you write the model to with the file extension '.model' (we will use another one when we just save the word vectors) 


In [35]:
model.save(fp+'.model')

It's then easy to reload the model:

In [36]:
from gensim.models import Word2Vec
tmp = Word2Vec.load(fp+'.model')
type(tmp), type(tmp.wv)

(gensim.models.word2vec.Word2Vec, gensim.models.keyedvectors.KeyedVectors)

#### Saving only the word vectors

Call the save `method()` on your word2vec `mode`'s `wv` attribute (a `KeyedVectors` object recording the model's word vectors):

**_Note:_** It's customary to name the file you write the model to with the file extension '.kv' 🥝


In [37]:
model.wv.save(fp+'.kv')

In [38]:
from gensim.models import KeyedVectors
tmp = KeyedVectors.load(fp+'.kv')
type(tmp)

gensim.models.keyedvectors.KeyedVectors

#### Save in other standard formats

Since Word2Vec has been implemented in many other languages (e.g., C and R), there is a standardized file format for storing word vectors.
Saving word embeddings in this format allows "interoperatbility" between languages. 
So for example, you could train your word embedding model in Python but compute with the embeddings in R.

In [39]:
model.wv.save_word2vec_format(fp+'.vectors')

The above code write a large text file.
We can read lines from it.


**_Note:_** To make the file smaller, set `binary=True` when calling `save_word2vec_format()`.

In [40]:
# inspect the file content (first two lines only)

max_ = 3
with open(fp+'.vectors') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= max_:
            break

79338 100
the 0.119308315 -0.040779013 0.1493203 0.09789107 0.21276306 0.0062533543 0.16182667 -0.06926643 0.27266514 0.049326796 -0.1469048 0.10646284 -0.13504066 0.5358513 0.3806534 -0.020854367 -0.09934769 -0.24901268 -0.04540069 0.055230558 0.1361154 0.21833535 -0.056823593 0.056802154 -0.35918522 0.24779136 0.060088005 0.07315052 0.11444866 0.023376534 0.19568448 -0.41516885 -0.27990136 0.0015731615 -0.04402603 0.27653602 -0.023319159 0.29740608 0.15914433 -0.131272 -0.009792739 -0.039231926 -0.4653306 -0.2246094 -0.067076266 0.035965934 -0.06107323 0.11750099 -0.2605005 -0.14819327 0.5176779 0.013491644 0.28505194 -0.09094592 0.07281482 0.46526077 0.06277853 -0.120132305 -0.3219086 0.060632497 0.2861146 -0.022832837 0.0871492 0.23346055 0.35255063 -0.19753844 -0.06287334 0.0040537957 -0.16039483 0.24120596 -0.048026964 -0.3107214 -0.028526496 0.32494506 0.22969867 0.40833458 -0.09506469 -0.6066375 -0.11606871 -0.08311282 0.05836072 0.1699447 -0.07935846 0.036100958 0.11423173 -0.